## Basic Installations and imports

In [ ]:
!pip install openai
!pip install tiktoken
!pip install langchain
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 114.7 MB/s eta 0:00:00


In [ ]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
import openai
import os
# Set up your OpenAI API key
os.environ['OPENAI_API_KEY'] = 'OPENAI_KEY'
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate
from langchain import PromptTemplate
from langchain.memory import ConversationBufferMemory

## Chunking

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/GP_CN_Fundamentals-merged.pdf")
pages = loader.load_and_split()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=50)
documents = text_splitter.split_documents(pages)

In [ ]:
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    llm=llm,
    output_key='answer',
    memory_key='chat_history',
    return_messages=True)

retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "include_metadata": True})

prompt_template = """

Answer the questions like a teacher would in a systematic way.
Context: \n {context}?\n
question: \n {question} \n

Answer:

"""
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

messages = [
            SystemMessagePromptTemplate.from_template(prompt_template),
]

qa_prompt = ChatPromptTemplate.from_messages(messages)

In [ ]:
chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  memory=memory,
  chain_type="stuff",
  retriever=retriever,
  return_source_documents=True,
  get_chat_history=lambda h : h,
  combine_docs_chain_kwargs={'prompt': qa_prompt},
  verbose=False)

In [ ]:
chat_history=[]

In [ ]:
query = "Hi there, how are you?"
result = chain({"question": query, "chat_history": chat_history})
result['answer']

"Hello! I'm doing well, thank you. How can I assist you today?"

In [ ]:
query = "What are the various layers in an OSI model?"
result = chain({"question": query, "chat_history": chat_history})
result['answer']

'The OSI (Open Systems Interconnection) model consists of seven layers. These layers are:\n\n1. Physical Layer: This layer is responsible for the physical transmission of data over the network. It deals with the electrical, mechanical, and physical aspects of the network, such as cables, connectors, and network interface cards.\n\n2. Data Link Layer: This layer is responsible for the reliable transmission of data between two directly connected nodes. It provides error detection and correction, flow control, and access control to the network.\n\n3. Network Layer: This layer is responsible for the routing of data packets across multiple networks. It determines the best path for data transmission and handles addressing and logical network topology.\n\n4. Transport Layer: This layer is responsible for the end-to-end delivery of data. It ensures that data is transmitted reliably and in the correct order. It also handles segmentation and reassembly of data.\n\n5. Session Layer: This layer es

The OSI (Open Systems Interconnection) model consists of seven layers. These layers are:

1. Physical Layer: This layer is responsible for the physical transmission of data over the network. It deals with the electrical, mechanical, and physical aspects of the network, such as cables, connectors, and network interface cards.

2. Data Link Layer: This layer is responsible for the reliable transmission of data between two directly connected nodes. It provides error detection and correction, flow control, and access control to the network.

3. Network Layer: This layer is responsible for the routing of data packets across different networks. It determines the best path for data transmission and handles addressing and logical network topology.

4. Transport Layer: This layer is responsible for the end-to-end delivery of data. It ensures that data is transmitted reliably and in the correct order. It also provides error detection and recovery mechanisms.

5. Session Layer: This layer establishes, manages, and terminates communication sessions between applications. It provides services such as session establishment, synchronization, and checkpointing.

6. Presentation Layer: This layer is responsible for the formatting and presentation of data. It translates data from the application layer into a format that can be understood by the network.

7. Application Layer: This layer provides services directly to the end-user applications. It includes protocols for email, file transfer, remote login, and other application-specific functions.

These layers work together to ensure that data is transmitted reliably and efficiently across a network. Each layer has its own specific functions and protocols, and they interact with each other through well-defined interfaces.

In [ ]:
query = "What are the differences between bus and star topology?"
result = chain({"question": query, "chat_history": chat_history})
result['answer']

'Bus Topology:\n- In a bus topology, all devices are connected to a single cable called the bus.\n- Devices are connected to the bus using T-connectors.\n- Data is transmitted in both directions on the bus.\n- If a device wants to transmit data, it broadcasts the data on the bus and all devices receive it.\n- Only one device can transmit data at a time, so collisions can occur if multiple devices try to transmit simultaneously.\n- If the bus cable is damaged or disconnected, the entire network will be affected.\n- Bus topologies are relatively simple and inexpensive to set up.\n\nStar Topology:\n- In a star topology, all devices are connected to a central hub or switch.\n- Each device has its own independent connection to the central hub/switch.\n- Data is transmitted from one device to another through the central hub/switch.\n- Only the intended recipient device receives the transmitted data.\n- If a device wants to transmit data, it sends the data to the central hub/switch, which the

Bus Topology:
- In a bus topology, all devices are connected to a single cable called the bus.
- Devices are connected to the bus using T-connectors.
- Data is transmitted in both directions on the bus.
- If a device wants to transmit data, it broadcasts the data on the bus and all devices receive it.
- Only one device can transmit data at a time, so collisions can occur if multiple devices try to transmit simultaneously.
- If the bus cable is damaged or disconnected, the entire network will be affected.
- Bus topologies are relatively simple and inexpensive to set up.

Star Topology:
- In a star topology, all devices are connected to a central hub or switch.
- Each device has its own independent connection to the central hub/switch.
- Data is transmitted from one device to another through the central hub/switch.
- Only the intended recipient device receives the transmitted data.
- If a device wants to transmit data, it sends the data to the central hub/switch, which then forwards it to the appropriate recipient.
- If a device or cable is damaged or disconnected, only that device will be affected and the rest of the network will continue to function.
- Star topologies require more cable compared to bus topologies, as each device needs its own connection to the central hub/switch.
- Troubleshooting in a star topology is relatively simple, as the central hub/switch acts as the aggregation point for all connected devices.

## Streamlit installation and imports

In [ ]:
!pip install streamlit streamlit_chat
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 115.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.522s

3 packages are looking for funding
  run `np

In [ ]:
%%writefile app.py

import streamlit as st
import openai
import os
# Set up your OpenAI API key
os.environ['OPENAI_API_KEY'] = 'OPENAI_KEY'
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate
from langchain import PromptTemplate
from streamlit_chat import message

st.set_page_config(page_title="NutriBot", page_icon=":smile:")

st.title('Welcome to (idk give it some name) bot!')

tab1= st.tabs(["📈 Talk Here"])

# from langchain.document_loaders import TextLoader  #for textfiles
# loader = TextLoader('/content/GP_CN_Fundamentals-merged.pdf')
# documents = loader.load()

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/GP_CN_Fundamentals-merged.pdf")
pages = loader.load_and_split()

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=50)
documents = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')

from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embeddings)

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    llm=llm,
    output_key='answer',
    memory_key='chat_history',
    return_messages=True)

retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "include_metadata": True})

prompt_template = """

Answer the questions like a teacher would in a systematic way.
If the query is outside this domain then do not answer it.
Context: \n {context}?\n
question: \n {question} \n

Answer:

"""
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

messages = [
            SystemMessagePromptTemplate.from_template(prompt_template),
]

qa_prompt = ChatPromptTemplate.from_messages(messages)

chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  memory=memory,
  chain_type="stuff",
  retriever=retriever,
  return_source_documents=True,
  get_chat_history=lambda h : h,
  combine_docs_chain_kwargs={'prompt': qa_prompt},
  verbose=False)

if "chat_history" not in st.session_state:
    st.session_state["chat_history"] = []
if "generated" not in st.session_state:
    st.session_state["generated"] = ["Hello ! Ask me anything about computer networks!"]
if "past" not in st.session_state:
    st.session_state["past"] = ["Hey ! 👋"]

#container for the chat history
response_container = st.container()
#container for the user's text input
container = st.container()

def generate_response(query):
    result = chain({"question": query, "chat_history": st.session_state["chat_history"]})
    #st.write(result['answer'])
    st.session_state["chat_history"] = [(query, result["answer"])]
    with st.sidebar:
        st.write(result['source_documents'])
    return result["answer"]

with container:
  with st.form(key="my_form", clear_on_submit=True):
      user_input = st.text_input("You:", key="input")
      submit_button = st.form_submit_button(label="Send")

      if user_input and submit_button:
        output = generate_response(user_input)
        #print(output)
        st.session_state["past"].append(user_input)
        st.session_state["generated"].append(output)
        st.session_state["chat_history"] = [(user_input, output)]

with response_container:
    for i in range(len(st.session_state['generated'])):
        message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="adventurer")
        message(st.session_state["generated"][i], key=str(i))

Overwriting app.py


In [ ]:
!curl https://ipv4.icanhazip.com/

34.41.38.4


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.289s
your url is: https://warm-sheep-share.loca.lt
